In [3]:
import pandas as pd
import numpy as np

# 10.1 GroupBy Mechanics
## split-apply-combine *Hadley Wickham*
*p.290*

In [12]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.785246,0.236659
1,a,two,-1.537695,0.134248
2,b,one,0.961192,-0.062640
3,b,two,0.603136,0.206945
4,a,one,-1.064376,0.004863


In [26]:
grouped = df['data1'].groupby(df['key1']).mean()

In [89]:
# grouped.count() par exemple

In [23]:
df['data1'].groupby([df['key1'],df['key2']]).mean()

key1  key2
a     one    -0.139565
      two    -1.537695
b     one     0.961192
      two     0.603136
Name: data1, dtype: float64

__Résultat :__
- dans le premier on prend la colonne data1 et regarde la moyenne pour chaque valeur de key1
- dans le deuxième on passe deux clé. Donc on a deux étages de clé. Séparation en key1 puis en key2

In [22]:
df['data1'].groupby([df['key2'],df['key1']]).mean()

key2  key1
one   a      -0.139565
      b       0.961192
two   a      -1.537695
      b       0.603136
Name: data1, dtype: float64

__Note :__ le résultat est commutatif pour le calcul de moyenne...

In [27]:
df['data1'].groupby([df['key2'],df['key1']]).size()

key2  key1
one   a       2
      b       1
two   a       1
      b       1
Name: data1, dtype: int64

In [36]:
# on veut un tableau non hierarchisé
df['data1'].groupby([df['key2'],df['key1']]).size().unstack()

key1,a,b
key2,,
one,2,1
two,1,1


et on obtient un dataframe xy :-)

## Iterating over groups

In [34]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.785246  0.236659
1    a  two -1.537695  0.134248
4    a  one -1.064376  0.004863
b
  key1 key2     data1     data2
2    b  one  0.961192 -0.062640
3    b  two  0.603136  0.206945


__Note :__ dans ce cas-ci name sont *a* et *b* alors que print(group) affiche l'id, key1, key2, data1 et data2 (les lignes quoi). Ici on n'a qu'une seule clé.

In [40]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.785246  0.236659
4    a  one -1.064376  0.004863
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.537695  0.134248
('b', 'one')
  key1 key2     data1    data2
2    b  one  0.961192 -0.06264
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.603136  0.206945


__Note :__ cette fois on a deux clés donc 2\*2 = 4 combinaisons possibles

In [49]:
liste = dict(list(df.groupby('key1')))

## Selecting a Column or Subset of Columns

In [71]:
# df.groupby('key1')['data1']
# df['data1'].groupby(df['key1'])
# both are the same

## Grouping with Dicts and Series

In [4]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns=['a','b','c','d','e'],
                     index=['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2:3,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,-1.373194,-0.008940,0.172194,0.672383,1.368784
Steve,-0.797416,0.316944,-0.083958,0.553091,0.611156
Wes,0.969447,NaN,NaN,2.231600,-1.260216
Jim,-0.422251,-1.193296,-0.962759,-0.370878,0.616626
Travis,0.619399,1.272207,0.176227,-0.998748,-0.478090


In [79]:
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
# ce qui change c'est le tableau de correspondance du nom des colonnes avec une couleur (non-unique)

In [83]:
people.groupby(mapping,axis=1).sum()

,blue,red
Joe,2.181689,1.042959
Steve,2.831307,2.378612
Wes,0.345241,-2.220943
Jim,-0.937318,0.090050
Travis,-3.693806,-1.700618


In [84]:
people.groupby(pd.Series(mapping), axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


Juste pour montrer que l'argument du groupby est un array mais aussi une série


__Trick :__ ctr+L donne le numéro de ligne dans Jupyter

## Grouping with Functions

In [18]:
people # dans l'index les noms ont 3, 5 ou 6 caractères

,a,b,c,d,e
Joe,-1.373194,-0.008940,0.172194,0.672383,1.368784
Steve,-0.797416,0.316944,-0.083958,0.553091,0.611156
Wes,0.969447,NaN,NaN,2.231600,-1.260216
Jim,-0.422251,-1.193296,-0.962759,-0.370878,0.616626
Travis,0.619399,1.272207,0.176227,-0.998748,-0.478090


In [20]:
people.groupby(len).sum() # on voit bien que au 6 on garde les nombres de travis et 5 aussi (il n'y a qu'un nom à 6 caractères, de même pour 5)

,a,b,c,d,e
3,-0.825998,-1.202236,-0.790565,2.533105,0.725194
5,-0.797416,0.316944,-0.083958,0.553091,0.611156
6,0.619399,1.272207,0.176227,-0.998748,-0.478090


In [38]:
(people.loc['Joe','a']+people.loc['Wes','a']+people.loc['Jim','a'])==people.groupby(len).sum().loc[3]['a']

True

__Remarque :__ on reconnaît comment est utilisé la fonction len (agit sur index) et la somme qui agit sur chaque retour similaire

#### On va essayer d'ajouter un masque

In [47]:
def lencustom(argument):
    longueur = len(argument)
    print(str(longueur)+' '+argument)
    return longueur
key_list = ['one','one','one','two','two']
people.groupby([lencustom,key_list]).min()
# premier regroupement puis séparation supplémentaire
# Jim inchangé car le seul sous 3 caractère ET two
# minimum de Wes et Joe

3 Joe
5 Steve
3 Wes
3 Jim
6 Travis


a         b         c         d         e
3 one -1.373194 -0.008940  0.172194  0.672383 -1.260216
  two -0.422251 -1.193296 -0.962759 -0.370878  0.616626
5 one -0.797416  0.316944 -0.083958  0.553091  0.611156
6 two  0.619399  1.272207  0.176227 -0.998748 -0.478090

## Grouping by Index Levels

In [56]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)

In [64]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [54]:
hier_df
# on a un masque avec us et jp puis un identifiant (non unique, une classe).
# Le masque est la ville et un deuxième étage

# on crée un data frame avec des chiffres randoms disponsés en 4 lignes, 5 colonnes

cty          US                            JP          
tenor         1         3         5         1         3
0      1.046334  0.716336 -0.420621 -0.328599  0.870906
1      1.195065 -1.092316 -0.279426  0.494057  0.643069
2      0.438987  1.591803 -0.794756  0.610802 -0.014194
3     -0.040395 -0.406340 -0.237547 -1.274025  1.148800

In [59]:
columns1 = pd.MultiIndex.from_arrays([['A','A','A','B','A'],['US','US','US','JP','JP'],[1,3,5,1,3]],names=['vaiante','cty','tenor'])
hier_df1 = pd.DataFrame(np.random.randn(4,5),columns=columns1)
hier_df1 # pas très joli avec la séparation...

vaiante         A                             B         A
cty            US                            JP        JP
tenor           1         3         5         1         3
0        0.813967  0.760396  0.510938  0.489203 -1.720852
1       -0.326163 -0.244631  2.192809 -0.753710  1.520874
2        1.021967 -0.062174 -1.709641 -0.870745  0.456485
3        1.692719  0.430738 -0.619499 -2.468621 -0.214636

In [63]:
hier_df1.transpose().sort_index().transpose() # c'est plus esthétique...

vaiante         A                                       B
cty            JP        US                            JP
tenor           3         1         3         5         1
0       -1.720852  0.813967  0.760396  0.510938  0.489203
1        1.520874 -0.326163 -0.244631  2.192809 -0.753710
2        0.456485  1.021967 -0.062174 -1.709641 -0.870745
3       -0.214636  1.692719  0.430738 -0.619499 -2.468621

# 10.2 Data Aggregation

Méthodes standards d'aggrégation :
- count
- sum
- mean
- median
- std, var
- min, max
- prod
- fisrt, last

In [137]:
'''datas = np.random.randn(5,4)
datas = datas.tolist()'''

'datas = np.random.randn(5,4)\ndatas = datas.tolist()'

In [138]:
'''for key,columns in enumerate(datas):
    columns[-2]=['a','a','b','b','a'][key]
    columns[-1]=['one','two','one','two','one'][key]
df = pd.DataFrame(datas,columns=['data1','data2','key1','key2'])
df'''

"for key,columns in enumerate(datas):\n    columns[-2]=['a','a','b','b','a'][key]\n    columns[-1]=['one','two','one','two','one'][key]\ndf = pd.DataFrame(datas,columns=['data1','data2','key1','key2'])\ndf"

In [142]:
df = pd.DataFrame(np.random.randn(5,2),columns=['data1','data2'])
df['key1'] = ['a','a','b','b','a']
df['key2'] = ['one', 'two', 'one', 'two', 'one']
df
# chemin beaucoup plus court !!!

,data1,data2,key1,key2
0,-0.433168,-0.773430,a,one
1,-1.508530,0.492091,a,two
2,-0.664450,-0.855185,b,one
3,-0.160106,-0.061311,b,two
4,-0.176519,0.243626,a,one


In [143]:
grouped = df.groupby('key1')
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)
# pour key1 a : intervalle pour data1 et data2 puis de même pour key1

,data1,data2
key1,,
a,1.332010,1.265521
b,0.504344,0.793875


## Column-Wise and Multiple Function Application

In [4]:
tips = pd.read_csv('../../data/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [147]:
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [150]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean') # on peut aussi passer nom de la fonction en string

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [154]:
grouped_pct.agg(['mean','std',peak_to_peak]) # marche pas si on met mean à la place de 'mean'

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Si on veut donner des noms :

In [157]:
grouped_pct.agg([('moyenne','mean'),('écart-type','std'),('recouvrement',peak_to_peak)])

moyenne  écart-type  recouvrement
day  smoker                                    
Fri  No      0.151650    0.028123      0.067349
     Yes     0.174783    0.051293      0.159925
Sat  No      0.158048    0.039767      0.235193
     Yes     0.147906    0.061375      0.290095
Sun  No      0.160113    0.042347      0.193226
     Yes     0.187250    0.154134      0.644685
Thur No      0.160298    0.038774      0.193350
     Yes     0.163863    0.039389      0.151240

__variation:__ groupement à deux étages

In [158]:
functions = ['count','mean','max']
grouped['tip_pct','total_bill'].agg(functions)

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

__variation :__ aggrégation différente pour chaque donnée

In [161]:
grouped.agg({'tip': np.max, 'size': 'sum'}) # différente

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [162]:
grouped.agg({'tip_pct' : ['min','max','mean','std'], 'size': 'sum'}) # différente et multiple

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## Returning Aggregated Data Without Row Indexes

In [164]:
tips.groupby(['day','smoker'], as_index=False).mean() # par d'index

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [172]:
df100 =pd.crosstab(tips.groupby(['day','smoker'], as_index=False).mean()['size'], tips.groupby(['day','smoker'], as_index=False).mean()['tip'])

# 10.3 Apply: General split-apply-combine

In [14]:
def top(df, n=5, columna='tip_pct'):
    return df.sort_values(by=columna)[-n:]
# entrée : dataframe, combien de valeurs à afficher, la colonne à sélectionner
# sortie : le même dataframe avec les n plus grands tip_pct, affiché par ordre coissant...

top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [17]:
tips.groupby('smoker').apply(top) # fait action une fois à yes et une fois à no. Puis concaténation.

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [18]:
tips.groupby(['smoker','day']).apply(top,n=1,columna='total_bill')
# les éléments suivants sont les arguments de la fonction
# top est appliqué à chaque groupe de smoker. mais pas aux sous groupes day ?

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [20]:
tips.groupby(['smoker','day']).describe() # cette fois-ci à tous les sous groupes

size                                                 tip  \
            count      mean       std  min  25%  50%   75%  max count   
smoker day                                                              
No     Fri    4.0  2.250000  0.500000  2.0  2.0  2.0  2.25  3.0   4.0   
       Sat   45.0  2.555556  0.784960  1.0  2.0  2.0  3.00  4.0  45.0   
       Sun   57.0  2.929825  1.032674  2.0  2.0  3.0  4.00  6.0  57.0   
       Thur  45.0  2.488889  1.179796  1.0  2.0  2.0  2.00  6.0  45.0   
Yes    Fri   15.0  2.066667  0.593617  1.0  2.0  2.0  2.00  4.0  15.0   
       Sat   42.0  2.476190  0.862161  1.0  2.0  2.0  3.00  5.0  42.0   
       Sun   19.0  2.578947  0.901591  2.0  2.0  2.0  3.00  5.0  19.0   
       Thur  17.0  2.352941  0.701888  2.0  2.0  2.0  2.00  4.0  17.0   

                       ...     tip_pct           total_bill             \
                 mean  ...         75%       max      count       mean   
smoker day             ...                                               
No     Fri   2.812500  ...    0.163652  0.187735        4.0  18.420000   
       Sat   3.102889  ...    0.183915  0.291990       45.0  19.661778   
       Sun   3.167895  ...    0.185185  0.252672       57.0  20.506667   
       Thur  2.673778  ...    0.184843  0.266312       45.0  17.113111   
Yes    Fri   2.714000  ...    0.209240  0.263480       15.0  16.813333   
       Sat   2.875476  ...    0.190502  0.325733       42.0  21.276667   
       Sun   3.516842  ...    0.215325  0.710345       19.0  24.120000   
       Thur  3.030000  ...    0.194837  0.241255       17.0  19.190588   

                                                               
                   std    min     25%     50%      75%    max  
smoker day                                                     
No     Fri    5.059282  12.46  15.100  19.235  22.5550  22.75  
       Sat    8.939181   7.25  14.730  17.820  20.6500  48.33  
       Sun    8.130189   8.77  14.780  18.430  25.0000  48.17  
       Thur   7.721728   7.51  11.690  15.950  20.2700  41.19  
Yes    Fri    9.086388   5.75  11.690  13.420  18.6650  40.17  
       Sat   10.069138   3.07  13.405  20.390  26.7925  50.81  
       Sun   10.442511   7.25  17.165  23.100  32.3750  45.35  
       Thur   8.355149  10.34  13.510  16.470  19.8100  43.11  

[8 rows x 32 columns]

In [22]:
result = tips.groupby('smoker').describe()
result

size                                                 tip            \
        count      mean       std  min  25%  50%  75%  max  count      mean   
smoker                                                                        
No      151.0  2.668874  1.017984  1.0  2.0  2.0  3.0  6.0  151.0  2.991854   
Yes      93.0  2.408602  0.810751  1.0  2.0  2.0  3.0  5.0   93.0  3.008710   

        ...     tip_pct           total_bill                             \
        ...         75%       max      count       mean       std   min   
smoker  ...                                                               
No      ...    0.185014  0.291990      151.0  19.188278  8.255582  7.25   
Yes     ...    0.195059  0.710345       93.0  20.756344  9.832154  3.07   

                                      
           25%    50%     75%    max  
smoker                                
No      13.325  17.59  22.755  48.33  
Yes     13.420  17.92  26.860  50.81  

[2 rows x 32 columns]

In [25]:
# result.unstack()

## Suppressing the Group Keys

In [24]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


# 10.4 Pivot Tables and Cross-Tabulation

In [48]:
res1 = tips.pivot_table(index=['day','smoker'])
res1

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [49]:
res2 = tips.groupby(['day','smoker']).mean()[['size','tip','tip_pct','total_bill']]
res2

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [51]:
res1.equals(res2)

True

In [56]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True)
# pour voir les effectifs

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

## Cross-Tabulations: Crosstab

In [57]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244